**All scripts and information necessary for reproducing the simulations present in this work is available here.**

Author: Valeria Añorve-Garibay

#### 1. Generate input

The length of the simulated region is **437,124 bp** with genetic structure from the modern human genome build GRCh37/hg2019. Defined coordinates: chr12: 40,547,438 - 40,984,562

I used the exon ranges defined by the GENCODE v.14 annotations, [Harrow et al. 2012](https://www.gencodegenes.org/human/release_14.html), which can be downloaded from UCSC Genome Browser using GENCODE Genes V14 Basic table.

In [1]:
import pandas as pd, numpy as np
from functools import reduce
import os.path, allel

In [2]:
# Function: sim_seq_file
# input: a gtf file containing just EXON information/ranges, coords start and coords end are the start
# and end of the region to simulate

def sim_seq_file(bedFile, coordsStart, coordsEnd, out):
    coords = pd.read_csv(bedFile, sep='\t', header=None)

    # exon ranges
    exonStarts = np.array(coords[1])[np.array(coords[1]) > coordsStart]
    exonEnds = np.array(coords[2])[np.array(coords[2]) > coordsStart]
    
    # create neutral ranges
    # i = 0
    neutralStarts = [coordsStart]
    neutralEnds = [exonStarts[0]-1]
    
    i = 1
    while i < len(exonStarts):
        neutralStarts.append(exonEnds[i-1]+1)
        neutralEnds.append(exonStarts[i]-1)
        i = i + 1

    # long final neutral region
    neutralStarts.append(exonEnds[i-1]+1)    
    neutralEnds.append(coordsEnd)
   
    exons = pd.DataFrame({'type': ['exon']*(len(exonStarts)), 'start': exonStarts, 'end': exonEnds})

    neutral = pd.DataFrame({'type': ['neutral']*(len(neutralStarts)), 'start': neutralStarts, 'end': neutralEnds})

    seq = (pd.concat([exons, neutral], axis=0, ignore_index=True).sort_values(by=['start'])).reset_index(drop=True)
    
    #seq.to_csv(out_op, sep=' ', index=False)
    # create genomic window based on the Neutral and Exon ranges
    # i = 0
    simStart = 0
    simEnd = seq['end'][0]-seq['start'][0]

    simStarts = [simStart]
    simEnds = [simEnd]

    i = 1
    while i < len(seq):
        simStart = simEnd+1
        simStarts.append(simStart)
        simEnd = simStart+(seq['end'][i]-seq['start'][i])
        simEnds.append(simEnd)
        i = i + 1

    simSeq = pd.DataFrame({'type': seq['type'], 'start': simStarts, 'end': simEnds})
    simSeq.to_csv(out, sep=' ', index=False)

In [3]:
# remove overlapping/duplicated exons using bedTools:
# https://davetang.org/muse/2013/01/18/defining-genomic-regions/
# https://github.com/davetang/defining_genomic_regions

#awk 'BEGIN{OFS="\t";} $3=="exon" {print $1,$4-1,$5}' MUC19_simRegion.gtf | \
#    bedtools2/bin/sortBed | bedtools2/bin/mergeBed -i - > MUC19_simRegion_exonRanges.gtf

In [4]:
sim_seq_file("data/MUC19_simRegion_exonRanges.bed", 40547438, 
             40984562, "data/sim_seq_MUC19.txt")

In [5]:
# Recombination map
recMap = pd.read_csv("data/MUC19_recombMap.bed", sep='\t', header=None)

In [6]:
recStart = recMap[2][0] - 40547438
recWindow = [recStart]

i = 1
while i < len(recMap)-1:
    recStart = recStart + (recMap[2][i] - recMap[1][i])
    recWindow.append(recStart)
    i = i + 1

recWindow.append(437124)

# stdrate must be multiplied by 1e-8 to convert it to cM/bp
simRecRates = pd.DataFrame({'type': ['recRate']*len(recWindow), 'pos': recWindow, 'stdrate': recMap[3]})
simRecRates.to_csv("data/sim_seq_MUC19.rmap", sep=' ', index=False)

**2. Run SLiM**

The SLiM simulation templates can be found in simulations/slim/

We obtained 25 simulation replicates for each of the different scenarios.

In [2]:
# Population branch statistic (PBS) per SNP
# Function: sim_pbs
# 1. Find the shared sites between the three samples (A, B and C)
# 2. Calculate PBS only in replicates with shared sites

# Population branch statistic (PBS) per SNP (combining all SNPs replicates)
# Function: sim_pbs
# 1. Use the MXL sites as reference
# 2. Find the shared sites between the three samples (A, B and C)
# 3. Calculate PBS
# input:
# pathA, pathB and pathC: VCF files
# nrep: number of replicates
# out: output file name

def compute_pbs(pathA, pathB, pathC, nrep, out):
    # define dictionaries
    pbs_dicc = {}
    position_dicc = {}
    rep_dicc = {}
    pO_dicc = {}
    mut_dicc = {}
    
    rep = 1
    
    while rep <= nrep:
        
        # if path exists
        path = pathA+str(rep)+'.txt'
        
        if os.path.isfile(path):
            # read vcf files
            popA = allel.read_vcf(pathA+str(rep)+'.txt', fields='*')
            popB = allel.read_vcf(pathB+str(rep)+'.txt', fields='*')
            popC = allel.read_vcf(pathC+str(rep)+'.txt', fields='*')
            
            popA_multiMask = popA['variants/MULTIALLELIC']
            popB_multiMask = popB['variants/MULTIALLELIC']
            popC_multiMask = popC['variants/MULTIALLELIC']

            popA_pos = popA['variants/POS'] 
            popB_pos = popB['variants/POS']
            popC_pos = popC['variants/POS']

            # shared sites between populations
            shared_sites = reduce(np.intersect1d, (popA_pos[~popA_multiMask], popB_pos[~popB_multiMask], popC_pos[~popC_multiMask]))
            
            if shared_sites.size == 0:
                print("Replicate", rep, "does not have shared sites")
                # fill in both dictionaries with Nans
                pbs_dicc[rep] = np.array([np.nan])
                position_dicc[rep] = np.array([np.nan])
                rep_dicc[rep] = np.array([np.nan])
                pO_dicc[rep] = np.array([np.nan])
                mut_dicc[rep] = np.array([np.nan])
            
            else:
                position_dicc[rep] = shared_sites
                rep_dicc[rep] = [rep]*len(shared_sites)
                
                # shared sites in each population
                popA_sites = np.isin(popA_pos, shared_sites)
                popB_sites = np.isin(popB_pos, shared_sites)
                popC_sites = np.isin(popC_pos, shared_sites)
                    
                # allele counts
                popA_countsArray = allel.GenotypeArray(popA['calldata/GT']).compress(popA_sites, axis=0).count_alleles()
                popB_countsArray = allel.GenotypeArray(popB['calldata/GT']).compress(popB_sites, axis=0).count_alleles()
                popC_countsArray = allel.GenotypeArray(popC['calldata/GT']).compress(popC_sites, axis=0).count_alleles()
                
                # calculate the numerator and denominator for Hudson's Fst estimator
                popA_popB_num, popA_popB_den = allel.hudson_fst(popA_countsArray, popB_countsArray)
                popA_popC_num, popA_popC_den = allel.hudson_fst(popA_countsArray, popC_countsArray)
                popC_popB_num, popC_popB_den = allel.hudson_fst(popC_countsArray, popB_countsArray)   
                
                # calculate Fst
                popA_popB_rawFst = popA_popB_num / popA_popB_den
                popA_popC_rawFst = popA_popC_num / popA_popC_den
                popC_popB_rawFst = popC_popB_num / popC_popB_den
                        
                # correct for negative Fst values
                popA_popB_Fst = np.where(popA_popB_rawFst < 0, 0, popA_popB_rawFst)
                popA_popC_Fst = np.where(popA_popC_rawFst < 0, 0, popA_popC_rawFst)
                popC_popB_fst = np.where(popC_popB_rawFst < 0, 0, popC_popB_rawFst)
                        
                # calculate PBS
                pbs = (((np.log(1.0 - popA_popB_Fst) * -1.0) +\
                        (np.log(1.0 - popA_popC_Fst) * -1.0) -\
                        (np.log(1.0 - popC_popB_fst) * -1.0)) / float(2))
                        
                pbs_dicc[rep] = pbs
                pOrigin = popA['variants/PO'].compress(popA_sites, axis=0)
                pMutType = popA['variants/MT'].compress(popA_sites, axis=0)
                pO_dicc[rep] = pOrigin
                mut_dicc[rep] = pMutType
    
        rep = rep + 1
    
    # concatenate dictionaries to create a data frame
    pbs_fullArray = []
    position_fullArray = []
    rep_fullArray = []
    pO_fullArray = []
    mutType_fullArray = []

    for i in list(pbs_dicc):
        pbs_fullArray = np.concatenate((pbs_fullArray, pbs_dicc[i][:]), axis = 0)
        position_fullArray = np.concatenate((position_fullArray, position_dicc[i][:]), axis = 0)
        rep_fullArray = np.concatenate((rep_fullArray, rep_dicc[i][:]), axis = 0)
        pO_fullArray = np.concatenate((pO_fullArray, pO_dicc[i][:]), axis = 0)
        mutType_fullArray = np.concatenate((mutType_fullArray, mut_dicc[i][:]), axis = 0)

    data = pd.DataFrame({'pbs':pbs_fullArray, 'position': position_fullArray,'pO': pO_fullArray, 'mt': mutType_fullArray, 'rep': rep_fullArray})
    data.to_csv(out, sep=' ', index=False)
    
    return data

In [3]:
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
# Uniform...
# Valeria's model
vnegative_u = compute_pbs(pathA = 'output/vnegative/uniform/VCF_MXL_negative_',
                   pathB = 'output/vnegative/uniform/VCF_EAS_negative_',
                   pathC = 'output/vnegative/uniform/VCF_EUR_negative_', nrep = 25,
                   out = "output/pbs/pbs_vnegative_uniform.txt")

vneutral_u = compute_pbs(pathA = 'output/vneutral/uniform/VCF_MXL_neutral_',
                   pathB = 'output/vneutral/uniform/VCF_EAS_neutral_',
                   pathC = 'output/vneutral/uniform/VCF_EUR_neutral_', nrep = 25,
                   out = "output/pbs/pbs_vneutral_uniform.txt")

In [5]:
# Santiago's model
snegative_u = compute_pbs(pathA = 'output/snegative/uniform/VCF_MXL_snegative_',
                   pathB = 'output/snegative/uniform/VCF_EAS_snegative_',
                   pathC = 'output/snegative/uniform/VCF_EUR_snegative_', nrep = 25,
                   out = "output/pbs/pbs_snegative_uniform.txt")

sneutral_u = compute_pbs(pathA = 'output/sneutral/uniform/VCF_MXL_sneutral_',
                   pathB = 'output/sneutral/uniform/VCF_EAS_sneutral_',
                   pathC = 'output/sneutral/uniform/VCF_EUR_sneutral_', nrep = 25,
                   out = "output/pbs/pbs_sneutral_uniform.txt")

In [6]:
# Recombination map
# Valeria's model
vnegative_r = compute_pbs(pathA = 'output/vnegative/map/VCF_MXL_negative_',
                   pathB = 'output/vnegative/map/VCF_EAS_negative_',
                   pathC = 'output/vnegative/map/VCF_EUR_negative_', nrep = 25,
                   out = "output/pbs/pbs_vnegative_map.txt")

vneutral_r = compute_pbs(pathA = 'output/vneutral/map/VCF_MXL_neutral_',
                   pathB = 'output/vneutral/map/VCF_EAS_neutral_',
                   pathC = 'output/vneutral/map/VCF_EUR_neutral_', nrep = 25,
                   out = "output/pbs/pbs_vneutral_map.txt")

In [7]:
# Santiago's model
snegative_r = compute_pbs(pathA = 'output/snegative/map/VCF_MXL_snegative_',
                   pathB = 'output/snegative/map/VCF_EAS_snegative_',
                   pathC = 'output/snegative/map/VCF_EUR_snegative_', nrep = 25,
                   out = "output/pbs/pbs_snegative_map.txt")

sneutral_r = compute_pbs(pathA = 'output/sneutral/map/VCF_MXL_sneutral_',
                   pathB = 'output/sneutral/map/VCF_EAS_sneutral_',
                   pathC = 'output/sneutral/map/VCF_EUR_sneutral_', nrep = 25,
                   out = "output/pbs/pbs_sneutral_map.txt")